In [1]:
#!pip install openpyxl

In [2]:
#!pip install pyspark

In [3]:
#!pip install pandasql

In [4]:
#!pip install duckdb

In [5]:
import sqlite3
import pandas as pd
import duckdb

In [19]:
print(f"sqlite3 version: {sqlite3.version}")
print(f"pandas version: {pd.__version__}")
print(f"duckdb version: {duckdb.__version__}")

sqlite3 version: 2.6.0
pandas version: 1.5.2
duckdb version: 0.7.1


Debería verse: 

sqlite3 version: 2.6.0

pandas version: 1.5.2

duckdb version: 0.7.1

In [6]:
# Cargar el archivo Excel en un DataFrame
df_CF = pd.read_excel('Prueba Técnica_Analista Calidad de Datos.xlsx', sheet_name='Country_Format')

# Imprimir los datos
print(df_CF.head())

  Country_Format Country_Code Country_Name  Format_Code    Format_Name
0           CR-1           CR   Costa Rica            1        Walmart
1           CR-2           CR   Costa Rica            2  Supermercados
2           CR-3           CR   Costa Rica            3        Bodegas
3           CR-5           CR   Costa Rica            5     Descuentos
4           GT-1           GT    Guatemala            1        Walmart


In [7]:
# Cargar el archivo Excel en un DataFrame
df_Stores = pd.read_excel('Prueba Técnica_Analista Calidad de Datos.xlsx', sheet_name='Stores')

# Imprimir los datos
print(df_Stores.head())

   Store_Code          Store_Name Country_Format
0         831     Wm Heredia Este           CR-1
1        4315            Wm Tibas           CR-1
2        3036         Mxm Heredia           CR-2
3        4424    Mxm Ciudad Colon           CR-2
4         793  Mp Turrialba Norte           CR-3


In [8]:
#Cargar el archivo Excel en un DataFrame
df_Sales = pd.read_excel('Prueba Técnica_Analista Calidad de Datos.xlsx', sheet_name='Sales')

# Imprimir los datos
print(df_Sales.head())

   Store_Code       Date  Quantity     Sales  Transactions
0          35 2023-01-01    8292.0  14319.83           509
1          35 2023-01-02   18072.0  31916.36           930
2          35 2023-01-03   18890.4  31089.59           962
3          35 2023-01-04   19178.4  32773.51          1045
4          35 2023-01-05   15756.0  28698.25           913


In [9]:
#Cargar el archivo Excel en un DataFrame
df_Sales_Plan = pd.read_excel('Prueba Técnica_Analista Calidad de Datos.xlsx', sheet_name='Sales_Plan')

# Imprimir los datos
print(df_Sales_Plan.head())

  Country_Format       Date    Sales_Plan
0           CR-1 2023-01-01  248141.20625
1           CR-1 2023-01-02  353650.15000
2           CR-1 2023-01-03  324767.40630
3           CR-1 2023-01-04  300620.20630
4           CR-1 2023-01-05  285410.30010


In [10]:
'''
import pandas as pd

# Realizar un join entre los data frames df_Sales, df_Stores y df_CF
df = pd.merge(df_Sales, df_Stores, on="Store_Code").merge(df_CF, on="Country_Format")

# Calcular los campos de Unidades Promedio, Transacciones Promedio y Precio Promedio
df["Unidades Promedio"] = df["Quantity"] / df["Transactions"]
df["Transacciones Promedio"] = df["Sales"] / df["Transactions"]
df["Precio Promedio"] = df["Sales"] / df["Quantity"]

# Seleccionar las columnas requeridas
result = df[["Country_Name", "Format_Name", "Store_Code", "Store_Name", "Date", 
             "Unidades Promedio", "Transacciones Promedio", "Precio Promedio"]]

# Agrupar por los campos requeridos
result = result.groupby(["Country_Name", "Format_Name", "Store_Code", "Store_Name", "Date"]) \
               .mean().reset_index()

# Mostrar el resultado
print(result)
'''

'\nimport pandas as pd\n\n# Realizar un join entre los data frames df_Sales, df_Stores y df_CF\ndf = pd.merge(df_Sales, df_Stores, on="Store_Code").merge(df_CF, on="Country_Format")\n\n# Calcular los campos de Unidades Promedio, Transacciones Promedio y Precio Promedio\ndf["Unidades Promedio"] = df["Quantity"] / df["Transactions"]\ndf["Transacciones Promedio"] = df["Sales"] / df["Transactions"]\ndf["Precio Promedio"] = df["Sales"] / df["Quantity"]\n\n# Seleccionar las columnas requeridas\nresult = df[["Country_Name", "Format_Name", "Store_Code", "Store_Name", "Date", \n             "Unidades Promedio", "Transacciones Promedio", "Precio Promedio"]]\n\n# Agrupar por los campos requeridos\nresult = result.groupby(["Country_Name", "Format_Name", "Store_Code", "Store_Name", "Date"])                .mean().reset_index()\n\n# Mostrar el resultado\nprint(result)\n'

In [11]:

# Realizar un join entre los data frames df_Sales, df_Stores y df_CF
df = pd.merge(df_Sales, df_Stores, on="Store_Code").merge(df_CF, on="Country_Format")

print(df.head(5))

   Store_Code       Date  Quantity     Sales  Transactions   Store_Name  \
0          35 2023-01-01    8292.0  14319.83           509  Paiz Petapa   
1          35 2023-01-02   18072.0  31916.36           930  Paiz Petapa   
2          35 2023-01-03   18890.4  31089.59           962  Paiz Petapa   
3          35 2023-01-04   19178.4  32773.51          1045  Paiz Petapa   
4          35 2023-01-05   15756.0  28698.25           913  Paiz Petapa   

  Country_Format Country_Code Country_Name  Format_Code    Format_Name  
0           GT-2           GT    Guatemala            2  Supermercados  
1           GT-2           GT    Guatemala            2  Supermercados  
2           GT-2           GT    Guatemala            2  Supermercados  
3           GT-2           GT    Guatemala            2  Supermercados  
4           GT-2           GT    Guatemala            2  Supermercados  


In [12]:


data_query = """
SELECT 
  Country_Name AS Pais, 
  Format_Name AS Formato, 
  Store_Code AS "Número de Tienda", 
  Store_Name AS "Nombre de la Tienda", 
  Date AS Fecha, 
  
  SUM(Quantity) / SUM(Transactions) AS "Unidades Promedio", 
  SUM(Sales) / SUM(Transactions) AS "Transacciones Promedio", 
  SUM(Sales) / SUM(Quantity) AS "Precio Promedio" 
FROM 
  df 
GROUP BY 
  Country_Name, 
  Format_Name, 
  Store_Code, 
  Store_Name, 
  Date
"""

df_result_1=duckdb.query(data_query).df() # returns a result dataframe

### Por lo tanto la tabla uno sería:

In [13]:
df_result_1

,Pais,Formato,Número de Tienda,Nombre de la Tienda,Fecha,Unidades Promedio,Transacciones Promedio,Precio Promedio
0,Guatemala,Supermercados,35,Paiz Petapa,2023-01-01,16.290766,28.133261,1.726945
1,Guatemala,Supermercados,35,Paiz Petapa,2023-01-02,19.432258,34.318667,1.766067
2,Guatemala,Supermercados,35,Paiz Petapa,2023-01-03,19.636590,32.317661,1.645788
3,Guatemala,Supermercados,35,Paiz Petapa,2023-01-04,18.352536,31.362211,1.708876
4,Guatemala,Supermercados,35,Paiz Petapa,2023-01-05,17.257393,31.432913,1.821417
...,...,...,...,...,...,...,...,...
3095,Costa Rica,Descuentos,4025,Pali Chacarita,2023-01-02,32.134490,49.973547,1.555137
3096,Costa Rica,Descuentos,4320,Pali Llorente,2023-01-27,22.140902,31.488548,1.422189
3097,Costa Rica,Bodegas,4318,Mp Mercedes Norte,2023-01-29,59.452654,102.741154,1.728117
3098,Costa Rica,Descuentos,3143,Pali Rio Frio,2023-01-25,25.960000,40.588042,1.563484


In [14]:
df2 = pd.merge(df, df_Sales_Plan, on='Country_Format')

In [15]:
#df2 = pd.merge(df, df_Sales_Plan, on='Country_Format')
#df2 = df2.rename(columns=lambda x: x + '_df' if x in df.columns else x + '_df_Sales_Plan')

In [16]:
df2

,Store_Code,Date_x,Quantity,Sales,Transactions,Store_Name,Country_Format,Country_Code,Country_Name,Format_Code,Format_Name,Date_y,Sales_Plan
0,35,2023-01-01,8292.0,14319.83,509,Paiz Petapa,GT-2,GT,Guatemala,2,Supermercados,2023-01-01,15209.90625
1,35,2023-01-01,8292.0,14319.83,509,Paiz Petapa,GT-2,GT,Guatemala,2,Supermercados,2023-01-02,37138.32500
2,35,2023-01-01,8292.0,14319.83,509,Paiz Petapa,GT-2,GT,Guatemala,2,Supermercados,2023-01-03,32109.08750
3,35,2023-01-01,8292.0,14319.83,509,Paiz Petapa,GT-2,GT,Guatemala,2,Supermercados,2023-01-04,33491.80625
4,35,2023-01-01,8292.0,14319.83,509,Paiz Petapa,GT-2,GT,Guatemala,2,Supermercados,2023-01-05,28158.15625
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96095,4411,2023-01-31,71208.0,181958.84,2419,Wm San Miguel,SV-1,SV,El Salvador,1,Walmart,2023-01-27,173806.67500
96096,4411,2023-01-31,71208.0,181958.84,2419,Wm San Miguel,SV-1,SV,El Salvador,1,Walmart,2023-01-28,229619.20630
96097,4411,2023-01-31,71208.0,181958.84,2419,Wm San Miguel,SV-1,SV,El Salvador,1,Walmart,2023-01-29,230688.93750
96098,4411,2023-01-31,71208.0,181958.84,2419,Wm San Miguel,SV-1,SV,El Salvador,1,Walmart,2023-01-30,187016.46250


In [17]:
data_query_2 = """
SELECT 
  Country_Name AS Pais, 
  Format_Name AS Formato, 
  Date_x AS Fecha, 
  1 - (Sales_Plan - Sales) / Sales AS "Alcance al Plan" 
FROM 
  df2
"""
df_result_2=duckdb.query(data_query_2).df() # returns a result dataframe


### Por lo tanto la tabla 2 sería:

In [18]:
df_result_2

,Pais,Formato,Fecha,Alcance al Plan
0,Guatemala,Supermercados,2023-01-01,0.937843
1,Guatemala,Supermercados,2023-01-01,-0.593489
2,Guatemala,Supermercados,2023-01-01,-0.242281
3,Guatemala,Supermercados,2023-01-01,-0.338841
4,Guatemala,Supermercados,2023-01-01,0.033625
...,...,...,...,...
96095,El Salvador,Walmart,2023-01-31,1.044802
96096,El Salvador,Walmart,2023-01-31,0.738071
96097,El Salvador,Walmart,2023-01-31,0.732192
96098,El Salvador,Walmart,2023-01-31,0.972205


In [21]:
df_result_2.dtypes

Pais                       object
Formato                    object
Fecha              datetime64[ns]
Alcance al Plan           float64
dtype: object

In [22]:
df_result_2['Fecha'] = pd.to_datetime(df_result_2['Fecha'], format='%d/%m/%Y')

### Guardar tablas finales (como archivos de Excel)

In [23]:
# Guardar df_result_1 como archivo de Excel
df_result_1.to_excel('ruta_del_archivo1.xlsx', index=False)

# Guardar df_result_2 como archivo de Excel
df_result_2.to_excel('ruta_del_archivo2.xlsx', index=False)